In [1]:
import os
from datetime import datetime
from src.python.classes.DBData import DBData
from src.python.classes.CallsClass import CallsClass
from duckdb import query

In [2]:
# carregando edições para filtro em queries
edicoes = DBData().ScrapDB(
    query_or_list_editions=2,
    edicao='Ponte'
)
edicoes

Edições carregadas.


,globalEvent
0,Desafio da Ponte


In [3]:
# consultando e exportando xlsx dos formulários e participantes p/ paths respectivos
conn_query = DBData().ScrapDB(
    query_or_list_editions=1,
    edicao='Desafio da Ponte',
    data_compra_ini='2022-05-01',
    data_compra_fini='2026-05-31',
    limit_max_rows=1000000,
    total_rows_in_batches=1000000
)


consulting ceps...
Total de registros carregados e exportados: 4547 ceps.

consulting participants...
Total de registros carregados e exportados: 4547 participants.
Forms e participants carregados e exportados.


In [4]:
# carregando xlsx de participantes em df
participants = CallsClass().load_and_parse_participants()

Arquivos encontrados:
 ['participants_results_Desafio_da_Ponte.xlsx']
Arquivo de participantes carregado.


In [5]:
# carregando xlsx de formulários em df
forms = CallsClass().load_and_parse_forms()

Arquivos encontrados:
 ['forms_results_Desafio_da_Ponte.xlsx']

Arquivo carregado. Parseando...
+------------------------------+------------------+
|                              | qtde (percent)   |
|------------------------------+------------------|
| total de ceps                | 4547 (100.00%)   |
| ceps com underline           | 20 (0.44%)       |
| ceps sem hifen               | 18 (0.40%)       |
| ceps ok                      | 4547 (100.00%)   |
| ceps com menos de 8 dígitos  | 0 (0.00%)        |
| ceps com mais de 8 dígitos   | 0 (0.00%)        |
| ceps com tratamento especial | 0 (0.00%)        |
+------------------------------+------------------+
Parseado com sucesso.


In [7]:
# Gets dos ceps no forms pela col 'parsed_ceps'. Do dict retornado, 'complete_api_df' contém os endereços.
gets = CallsClass().triforce(
    parsed_ceps_df=forms,
    timeout=5
)
addresses_df = gets.get('complete_api_df')

100%|██████████| 10/10 [01:04<00:00,  7.28s/it]

+------+-------+
|   ok |   nok |
|------+-------|
|    6 |     4 |
+------+-------+
+---------------------+-----------------+-----------------+
|   brasil api errors |   viacep errors |   apicep errors |
|---------------------+-----------------+-----------------|
|                   4 |               4 |               4 |
+---------------------+-----------------+-----------------+


100%|██████████| 10/10 [01:09<00:00,  6.93s/it]


In [8]:
# check
addresses_df.head()

,item_id,cep,state,city,neighborhood,street,service
0,451708,22441140,RJ,Rio de Janeiro,Leblon,Rua General Artigas,open-cep
1,451711,28895242,RJ,Rio das Ostras,Costazul,Rua Prefeito Wilson de Barros Vieira,open-cep
2,451712,22471003,RJ,Rio de Janeiro,Lagoa,Avenida Epitácio Pessoa,open-cep
3,451713,22725031,RJ,Rio de Janeiro,Taquara,Estrada do Cafundá,open-cep
4,451715,22431002,RJ,Rio de Janeiro,Leblon,Avenida Bartolomeu Mitre,open-cep


In [8]:
# gerando df para export
participants_and_addresses = query(
    query='''
    SELECT 
      participants.*
    , addresses_df.cep as cep
    , addresses_df.state as UF
    , addresses_df.city as cidade
    , addresses_df.neighborhood as bairro
    , addresses_df.street as logradouro
    FROM participants
    LEFT JOIN addresses_df
    ON participants.itemID = addresses_df.item_id
    '''   
).to_df()

In [9]:
# exportando df resultante do evento para xlsx

folder = os.path.join(os.getcwd(), 'src', 'datasets', 'results')
os.makedirs(folder, exist_ok=True)

edition = participants['edição'].unique()[0].replace(' ','_')
current_time = datetime.now().strftime("%Y%m%d_%H%M")

result_df_path = os.path.join(folder, f'{edition}_{current_time}.xlsx')
participants_and_addresses.to_excel(result_df_path, index=False)

print(f'Arquivo {edition}_{current_time}.xlsx exportado.\npath: {result_df_path}')

Arquivo Desafio_da_Ponte_20250806_0043.xlsx exportado.
path: c:\Users\vitor\Desktop\ceps\src\datasets\results\Desafio_da_Ponte_20250806_0043.xlsx
